Klassifikationsmodell bauen, mit dem Städte auf der Grundlage des PM2.5-Werts in zwei Gruppen eingeteilt werden.

Mehrstufige Erarbeitung:

Schwellenwerte
1. WHO-Richtline (fachlicher Standard --> internationale Vergleichbarkeit)
2 Median oder Perzentile (datengetrieben --> Mehrwert von Data Science)

Modelle
1. Logistische Regression
2. Random Forest
3. Gradient Boosting



In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

In [ ]:
# Liste relevanter Schadstoffe
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

# Mittelwerte pro Stadt berechnen (Index = City)
df_means = df.groupby('City')[pollutants].mean()


## 🧾 Definition der Zielvariable AirQualityLabel

Um ein Klassifikationsmodell zur Vorhersage der Luftqualität von Städten zu erstellen, wurde eine zielvariable AirQualityLabel eingeführt. Diese ordnet jeder Stadt eine von zwei Klassen zu:

- 0 → Gute Luftqualität

- 1 → Schlechte Luftqualität

Die Einteilung basiert auf dem Medianwert der durchschnittlichen PM2.5-Konzentration aller Städte im Datensatz. Der Median wurde als datengetriebener Grenzwert gewählt, da sich der offiziell empfohlene WHO-Grenzwert von 5 µg/m³ in der Praxis als zu streng erwiesen hat: Nur drei Städte hätten diesen erfüllt, was zu einem extremen Klassenungleichgewicht und damit zu einem ungeeigneten Klassifikationsproblem geführt hätte.

Durch die Verwendung des Medians entsteht eine ausgewogene Verteilung zwischen den beiden Klassen, die ein stabiles Training und eine faire Bewertung des Modells ermöglicht.

Die Berechnung erfolgte folgendermaßen:

    pm25_median = df_means['Pm25'].median()
    df_means['AirQualityLabel'] = (df_means['Pm25'] > pm25_median).astype(int)

 Städte mit einem PM2.5-Mittelwert über dem Median wurden als "schlechte Luftqualität" (1) klassifiziert, alle anderen als "gute Luftqualität" (0).



In [ ]:
# Median von PM2.5 berechnen
pm25_median = df_means['Pm25'].median()

# Zielvariable hinzufügen
df_means['AirQualityLabel'] = (df_means['Pm25'] > pm25_median).astype(int)

In [ ]:
df_means.head()

In [ ]:
# Anzahl Städte mit guter/schlechter Luft (nach WHO-Grenzwert)
(df_means['Pm25'] <= 5).value_counts()

In [ ]:
# Städte mit PM2.5 ≤ 5 µg/m³ filtern
clean_cities = df_means[df_means['Pm25'] <= 5]

# Ergebnis anzeigen
clean_cities

In [ ]:
# Anzahl gültiger PM2.5-Werte pro Stadt
pm25_counts = df.groupby('City')['Pm25'].count().sort_values()

# Zeige nur die "sauberen" Städte
pm25_counts.loc[['Plovdiv', 'Yazd', 'Zürich']]

In [ ]:
import matplotlib.pyplot as plt

# Nur gültige PM2.5-Werte für Yazd
yazd_pm25 = df[(df['City'] == 'Yazd') & (df['Pm25'].notna())]

# Histogramm der Jahresverteilung
plt.figure(figsize=(8, 4))
yazd_pm25['Month'].value_counts().sort_index().plot(kind='bar', color='steelblue')

plt.title("Anzahl gültiger PM2.5-Messwerte pro Jahr für Yazd")
plt.xlabel("Jahr")
plt.ylabel("Anzahl gültiger Messwerte")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# Liste problematischer Städte
rauswerfen = ['Plovdiv', 'Yazd']

# Entferne sie aus df und ggf. df_means
df = df[~df['City'].isin(rauswerfen)]

# Und aus dem aggregierten Mittelwert-Datensatz
df_means = df_means[~df_means.index.isin(rauswerfen)]

Sehr schnelles Zwischenergebnis:

Es gibt nur 3 Städte, die den aktuellen WHO-Anspruch von weniger als 5 µg/m³ erfüllen: Zürich (Schweiz), Plovdiv (Bulgarien), Yazd (Iran). Bei genauerem Hinschauen fällt zusätzlich auf, dass es für Plovidiv nur enen einzigen Messwert gibt, und für Yazd nur sehr wenige. 
Die WHO-Richtline kann also nicht verwendet werden, weil ein Modell damit nicht trainierbar ist - es hat keine zweite Klasse.

Wir nehmen also direkt den Median, gehen also datengetrieben vor.

In [ ]:
# Umgang mit NaN-Werten

# Anzahl fehlender Werte pro Spalte
df[["City"] + pollutants].isna().sum()

In [ ]:
# Anzahl gültiger Werte pro Stadt und Schadstoff
df.groupby('City')[pollutants].count().sort_values(by='Pm25')


In [ ]:
# Welche Städte enthalten überhaupt keine Schadstoffwerte?

# Alle Schadstoffspalten
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

# Für jede Stadt zählen, wie viele gültige Werte es insgesamt gibt
valid_counts = df.groupby('City')[pollutants].count()

# Zeige nur Städte mit 0 gültigen Werten in *allen* Schadstoffspalten
no_data_cities = valid_counts[(valid_counts == 0).all(axis=1)]

# Ausgabe
no_data_cities.index.tolist()

In [ ]:
no_data_cities = [
    'Alor setar', 'George town', 'Ipoh', 'Johor bahru', 'Klang', 'Kota bharu',
    'Kuala lumpur', 'Kuantan', 'Kuching', 'Malacca', 'Miri', 'Seremban', 'Taiping'
]
# Entferne sie aus df und ggf. df_means

# Aus dem Haupt-DataFrame
df = df[~df['City'].isin(no_data_cities)]

# Auch aus df_means entfernen (falls schon berechnet)
df_means = df_means[~df_means.index.isin(no_data_cities)]

In [ ]:
df_means.shape

In [ ]:
df_means.head()

In [ ]:
# Relevante Spalten
pollutants = ['Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2']

# Für jede Stadt: Wie viele Mittelwerte sind vorhanden?
df_means['Num_Valid_Pollutants'] = df_means[pollutants].notna().sum(axis=1)

# Übersicht: Wie viele Städte haben wie viele gültige Schadstoffwerte?
coverage_summary = df_means['Num_Valid_Pollutants'].value_counts().sort_index()

# Ergebnis anzeigen
coverage_summary


Weil das Imputieren von überhaupt nicht vorhandenen Kategorien (Schadstoffen) für eine Stadt eigentlich nicht mehr als "Raten" ist, machen wir das hier nicht und reduzieren die trainingsdaten auf die 404 Städte, die für alle Schadstoffe Werte gemeldet haben.

In [ ]:
# Nur Städte mit allen 6 Schadstoff-Mittelwerten
df_means_complete = df_means[df_means['Num_Valid_Pollutants'] == 6]

In [ ]:
df_means_complete.head()

In [ ]:
df_means_complete.shape

In [ ]:
features = ['Co', 'No2', 'O3', 'Pm10', 'So2']
X = df_means_complete[features]
y = df_means_complete['AirQualityLabel']

In [ ]:
# Wir haben  aktuell 404 Städte in df_means_complete. Davon nehmen wir 80% für das Training und 20% für den Test.
# Wir verwenden den Standard-Trainings-Test-Split von scikit-learn.

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modell 1: Logistic Regression

In [ ]:

from sklearn.linear_model import LogisticRegression

#  Modelltraining
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Wenn wir PM10 als Feature drinlassen, klassifiziert das Modell prima - und das ist auch zu erwarten, weil PM2.5 eine Teilmenge von PM10 ist. Also lassen wir PM10 jetzt mal raus und schauen, was dann passiert:

In [ ]:
features_no_PM10 = ['Co', 'No2', 'O3', 'So2']
X = df_means_complete[features_no_PM10]
y = df_means_complete['AirQualityLabel']

In [ ]:
# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelltraining
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


## 🧾 Entscheidung zur Feature-Auswahl: PM10 aus den Features entfernen

Um die Luftqualität in Städten zu klassifizieren, wurden zunächst alle Schadstoffe (CO, NO₂, O₃, PM10, PM2.5, SO₂) als Features in das Modell aufgenommen, obwohl PM10 und PM2.5 naturgemäß eine hohe Korrelation aufweisen (über 97%).

🔹 Test mit PM10 als Feature:

Das Modell erzielte eine hohe Genauigkeit von 91% und einen sehr hohen F1-Score von 0.92 für die Vorhersage von schlechter Luft (Klasse 1).

Präzision und Recall bei der Klassifikation von „schlechter Luft“ waren sehr hoch (nahe 1.0), was das Modell besonders präzise bei der Vorhersage von schlechter Luft machte.

Allerdings zeigte sich auch, dass das Modell zwischen den Klassen kaum differenzierte, was die klassenspezifische Leistung bei guter Luft (Klasse 0) beeinträchtigte.

🔹 Test ohne PM10 als Feature:

Nachdem PM10 entfernt wurde, ging die Genauigkeit leicht zurück auf 80% und der F1-Score für schlechte Luft sank von 0.92 auf 0.83.

Das Modell zeigt nun jedoch eine ausgewogenere Klassifikation (bevorzogt nicht mehr das Label "schlechte Luft") und ist weniger von redundanten Informationen beeinflusst.

🧠 Ergebnis:

Die Tests haben gezeigt, dass das Modell besser differenziert, wenn PM10 aus den Features entfernt wird. Auch wenn die Gesamtgenauigkeit etwas zurückgeht, ist das Modell nun besser in der Lage, zwischen guter und schlechter Luft zu unterscheiden.
Die Entscheidung, PM10 aus den Features zu entfernen, basiert auf der starken Korrelation mit PM2.5, die zu einer Redundanz führte und das Modell verzerrte.



## Modell 2: Random Forest

In [ ]:
# Features and train test split repeated from previous model, just to make clear what is being used:

features_no_PM10 = ['Co', 'No2', 'O3', 'So2']
X = df_means_complete[features_no_PM10]
y = df_means_complete['AirQualityLabel']

# Split in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest-Modell erstellen
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Modell trainieren
rf_model.fit(X_train, y_train)


In [ ]:
# Vorhersagen & Bewertung drucken
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
# Feature importance extrahieren
importances = rf_model.feature_importances_
len(importances)

In [ ]:


# Feature-Importanz in ein DataFrame umwandeln
feature_importance_df = pd.DataFrame({
    'Feature': features_no_PM10,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Feature-Importanz anzeigen
plt.figure(figsize=(8, 4))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()



Am wichtigsten für die Klassifizierung ist das Feature "CO". Kann es sein, dass die flasch klassifizierten Städte auffällige CO-Werte haben?

In [ ]:
# Falsch klassifizierte Städte finden
incorrect_predictions = X_test.copy()
incorrect_predictions['True Label'] = y_test
incorrect_predictions['Predicted Label'] = y_pred_rf

# Nur falsch klassifizierte Städte herausfiltern
incorrect_predictions = incorrect_predictions[incorrect_predictions['True Label'] != incorrect_predictions['Predicted Label']]

# CO-Werte der falsch klassifizierten Städte
incorrect_predictions['CO'] = X_test.loc[incorrect_predictions.index, 'Co']

# Ausgabe der Städte mit ihren CO-Werten
incorrect_predictions[['True Label', 'Predicted Label', 'CO']]


In [ ]:
# CO-Werte der falsch klassifizierten Städte
co_values = incorrect_predictions['CO']

# Berechne die wichtigsten Statistiken (Durchschnitt, IQR)
co_mean = co_values.mean()
co_std = co_values.std()
co_min = co_values.min()
co_max = co_values.max()

# Berechne Interquartilsabstand (IQR)
Q1 = co_values.quantile(0.25)
Q3 = co_values.quantile(0.75)
IQR = Q3 - Q1

# Anzeigen der CO-Statistiken
print(f"Durchschnittlicher CO-Wert: {co_mean:.2f}")
print(f"Standardabweichung: {co_std:.2f}")
print(f"Minimaler CO-Wert: {co_min:.2f}")
print(f"Maximaler CO-Wert: {co_max:.2f}")
print(f"Interquartilsabstand (IQR): {IQR:.2f}")


Hier muss man noch nachdenken, wie man das Modell verbessern kann. Man sollte die falsch zugeordneten Städte überprüfen, ob da mit den Daten was nicht stimmt. Und dann noch die Hyperparameter besser einstellen und so. Nicht mehr heute.

## Modell 3: Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Gradient Boosting Modell erstellen
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Modell trainieren
gb_model.fit(X_train, y_train)

# Vorhersagen & Bewertung
y_pred = gb_model.predict(X_test)
print(classification_report(y_test, y_pred_gb))
print(confusion_matrix(y_test, y_pred_gb))
